In [ ]:
from pysidt import *
from pysidt.extensions import split_mols
from pysidt.sidt import *
import json
import numpy as np
from molecule.molecule import *
from molecule.molecule.atomtype import ATOMTYPES
from sklearn import linear_model
import scipy.sparse as sp
import logging
import random
import itertools
import os
from pysidt.plotting import *
from pynta.mol import *
from pynta.geometricanalysis import *
from pynta.coveragedependence import mol_to_atoms, process_calculation, adsorbate_interaction_decomposition
from pynta.postprocessing import *
from ase.visualize import view
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#ammonia decomp
path = "/Users/mjohns9/Runs/coveragedep/Pt111_ammonia_N_1_8_25"
pynta_path = "/Users/mjohns9/Runs/pynta/Pt111_ammonia_decomp_10_2_24"
rxn_file = "/Users/mjohns9/Runs/pynta/Pt111_ammonia_decomp_10_2_24/ammonia_rxns.yaml"
slab_path = "/Users/mjohns9/Runs/pynta/Pt111_ammonia_decomp_10_2_24/slab.xyz"
ts_dict = {"TS1":"22", "TS2": "37", "TS4": "98","TS7": "25"}
coad_names = ["N#[Pt]"]
coad_nice_names = {"N#[Pt]":"N"}
cmap = plt.colormaps["Blues"]
MLsize = 9 #number of site occupations that constitute one monolayer (for (a,b,c) size slab this is usually a*b)
to_eV = 1.0/(96.48530749925793*1000.0) #converts from J/mol to eV

metal = "Pt"
facet = "fcc111"
nslab = 36
slabxyz = os.path.join(pynta_path,"slab.xyz")
slab = read(slabxyz)
cas = SlabAdsorptionSites(slab,facet,allow_6fold=False,composition_effect=False,
                        label_sites=True,
                        surrogate_metal='Pt')

sites = cas.get_sites()
conn = cas.get_connectivity()
site_adjacency = cas.get_neighbor_site_list()

In [ ]:
#Extract data
Ncoad_energy_dict,Ncoad_config_dict,tree_dict,admol_name_structure_dict,admol_name_path_dict,ts_info,max_coad_indexes,ad_energy_dict,coadmol_E_dict = extract_covdep_data(
                                                                    path,pynta_path,ts_dict,metal,facet,sites,site_adjacency,nslab,coad_names)
 

In [ ]:
#analyze an individual configuration
config_name = 'N#[Pt]'
coad_name = 'N#[Pt]'
coad_nice_name = coad_nice_names[coad_name]
forward = True
if config_name in ts_dict.keys():
    if forward:
        reactant_names = ts_info[config_name]['species_names']
    else:
        reactant_names = ts_info[config_name]['reverse_names']
else:
    reactant_names = None
        
L = len(Ncoad_energy_dict[coad_name])
lowest_energy_configs = []
leg = []
for i in sorted(Ncoad_energy_dict[coad_name].keys()):
    leg.append("SIDT Iter "+str(i))
    Ncoads = np.array(sorted([x for x in Ncoad_energy_dict[coad_name][i][config_name].keys() if x-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys()]))
    energies = [get_energy_correction_configuration(Ncoad_energy_dict,ts_dict,config_name,coad_name,i,Ncoad,reactant_names=reactant_names)*to_eV if Ncoad > 0 else 0.0 for Ncoad in Ncoads if Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys()]
    plt.plot(Ncoads/MLsize,energies,color=cmap(i/L))

plt.legend(leg,loc="upper left")
if config_name not in ts_dict.keys():
    plt.ylabel("Energy Correction " + config_name + " [eV]")
else:
    plt.ylabel("Energy Correction TS " + ts_info[config_name]["name"] + " [eV]")
#plt.ylabel("Energy Correction NH3X [eV]")
plt.xlabel(r"$\theta_{"+coad_nice_name+r"}$")



In [ ]:
#Fit configuration energy correction (This cell draws parameters from the first individual configuration cell!)
L = len(Ncoad_energy_dict)-1
coverages = np.array([0.0]+sorted([x for x in Ncoad_energy_dict[coad_name][i][config_name].keys() if x-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys()]))/MLsize
energies = [0.0]+[get_energy_correction_configuration(Ncoad_energy_dict,ts_dict,config_name,coad_name,i,Ncoad,reactant_names=reactant_names)*to_eV if Ncoad > 0 else 0.0 for Ncoad in Ncoads if Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys()]

params1 = np.polyfit(coverages,energies,1)
params2 = np.polyfit(coverages,energies,2)
params3 = np.polyfit(coverages,energies,3)
params4 = np.polyfit(coverages,energies,4)
params5 = np.polyfit(coverages,energies,5)
params6 = np.polyfit(coverages,energies,6)

vals1 = np.polyval(params1,coverages)
vals2 = np.polyval(params2,coverages)
vals3 = np.polyval(params3,coverages)
vals4 = np.polyval(params4,coverages)
vals5 = np.polyval(params5,coverages)
vals6 = np.polyval(params6,coverages)

plt.scatter(coverages,energies)
plt.plot(coverages,vals1)
plt.plot(coverages,vals2)
plt.plot(coverages,vals3)
plt.plot(coverages,vals4)
plt.plot(coverages,vals5)
plt.plot(coverages,vals6)
plt.legend(["SIDT","linear","2nd order","3rd order","4th order","5th order","6th order"])
plt.ylabel("Species Energy Correction " + config_name + " [eV]")
plt.xlabel(r"$\theta_{"+coad_nice_name+r"}$")

plt.figure()
poly_maes = [np.mean(np.abs(energies-vals1)),np.mean(np.abs(energies-vals2)),
                     np.mean(np.abs(energies-vals3)),np.mean(np.abs(energies-vals4)),
                    np.mean(np.abs(energies-vals5)),np.mean(np.abs(energies-vals6))]
plt.plot(range(1,7),poly_maes,'o-')

plt.xticks(range(1,7))
plt.ylim(0.0,np.max(poly_maes)+0.1*np.max(poly_maes))
plt.ylabel("Fitting MAE [eV]")
plt.xlabel("Polynomial Order")

#print(f"Polynomial coefficients (highest to lowest order): {params3}") #paramsN corresponds to the Nth order polynomial


In [ ]:
#visualize lowest energy (2D) configurations (in 3D) (This cell draws parameters from the first individual configuration cell!)
#Note that there may be more than one predicted lowest energy configuration at each coverage and this extracts all of them

iter_configs = len(Ncoad_energy_dict)-1 #which iteration to look at

configs_3D,configs_2D,sidt_Es,sidt_traces = analyze_covdep_lowest_energy(Ncoad_config_dict,iter_configs,config_name,
                                                coad_name,metal,slab,sites,admol_name_structure_dict,admol_name_path_dict,tree_dict)

#view(configs_3D) #see configurations in 3D
#print({i+1:E for i,E in enumerate(sidt_Es)}) #SIDT predicted energies of each configuration
#print({i+1:{(n,i):tree_dict[iter_configs].nodes[n].rule.value for i,n in enumerate(tr)} for i,tr in enumerate(sidt_traces)}) #Final tree nodes and their final rule value associated with each prediction
#print({i+1:m.to_adjacency_list() for i,m in enumerate(configs_2D)}) #see configurations in 2D (adjacency lists)

In [ ]:
#visualize sampled data (calculated 3D geometries) (This cell draws parameters from the first individual configuration cell!)

reactant_names = None #for TSs the energy correction depends on the direction of the reaction and the list of reactant names needs specified here

configs_3D,config_Es,config_E_correction,config_xyzs,config_mols = analyze_covdep_sample_data(config_name,coad_name,
                                                                Ncoad_energy_dict,path,pynta_path,slab,metal,facet,sites,site_adjacency,
                                                                        ad_energy_dict,ts_dict,coadmol_E_dict,reactant_names=reactant_names)

#view(configs_3D) #Visualize the configurations in 3D
#print({i+1:E for i,E in enumerate(config_Es)}) #Association energies of each configuration
#print({i+1:E for i,E in enumerate(config_E_correction)}) #Energy corrections (energy to move adsorbate/TS (from isolation) and coadsorbates (at coverage) into the configuration)
#print({i+1:config_xyzs[i] for i,E in enumerate(config_Es)}) #location of the xyz files for each configuration
#print({i+1:m.to_adjacency_list() for i,m in enumerate(config_mols)}) #2D adjacency lists for each configuration

In [ ]:
#analyze a reaction 
ts_name = "TS7"
coad_name = "N#[Pt]"
forward = True
if forward:
    reactant_names = ts_info[ts_name]['species_names']
    product_names = ts_info[ts_name]['reverse_names']
else:
    reactant_names = ts_info[ts_name]['reverse_names']
    product_names = ts_info[ts_name]['species_names']

rstr = "+".join(reactant_names) + "=>" + "+".join(product_names)
print(rstr)
reactant_energies = {i: {k : sum([get_energy_correction_configuration(Ncoad_energy_dict,ts_dict,name,coad_name,i,k) if k != 0 else 0.0 for name in reactant_names if name in Ncoad_energy_dict[coad_name][i].keys()]) for k in Ncoad_energy_dict[coad_name][i][ts_name].keys() if all(k in Ncoad_energy_dict[coad_name][i][name] for name in reactant_names if name in Ncoad_energy_dict[coad_name][i].keys())} for i in Ncoad_energy_dict[coad_name].keys()}
product_energies = {i: {k : sum([get_energy_correction_configuration(Ncoad_energy_dict,ts_dict,name,coad_name,i,k) if k != 0 else 0.0 for name in product_names if name in Ncoad_energy_dict[coad_name][i].keys()]) for k in Ncoad_energy_dict[coad_name][i][ts_name].keys() if all(k in Ncoad_energy_dict[coad_name][i][name] for name in product_names if name in Ncoad_energy_dict[coad_name][i].keys())} for i in Ncoad_energy_dict[coad_name].keys()}

In [ ]:
#barrier height (This cell draws parameters from the first reaction cell!)
L = len(Ncoad_energy_dict[coad_name])-1
leg = []
for i in sorted(Ncoad_energy_dict[coad_name].keys()):
    leg.append("SIDT Iter "+str(i))
    Ncoads = np.array([Ncoad for Ncoad in sorted(Ncoad_energy_dict[coad_name][i][ts_name].keys()) if Ncoad == 0 or (Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys() and Ncoad in Ncoad_energy_dict[coad_name][i][ts_name].keys() and all(Ncoad in Ncoad_energy_dict[coad_name][i][n].keys() for n in reactant_names if n in Ncoad_energy_dict[coad_name][i].keys()))])
    energies = [get_barrier_correction(Ncoad_energy_dict,ts_dict,ts_name,coad_name,i,Ncoad,reactant_names)*to_eV if Ncoad != 0 else 0.0 for Ncoad in Ncoads]
    plt.plot(Ncoads/MLsize,energies,color=cmap(i/L))

plt.legend(leg,loc = "upper left")
plt.ylabel("Barrier Correction " + rstr + " [eV]")
#plt.ylabel("Barrier Correction NH3X => HX + NH2X [eV]")
plt.xlabel(r"$\theta_{"+coad_nice_name+r"}$")
#plt.ylim(-2.0,3.0)
#plt.xlim(0,0.65)

In [ ]:
#analyze reaction energy (This cell draws parameters from the first reaction cell!)
L = len(Ncoad_energy_dict[coad_name])-1
leg = []
for i in sorted(Ncoad_energy_dict[coad_name].keys()):
    leg.append("SIDT Iter "+str(i))
    Ncoads = np.array([Ncoad for Ncoad in sorted(Ncoad_energy_dict[coad_name][i][ts_name].keys()) if Ncoad == 0 or (Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys() and all(Ncoad in Ncoad_energy_dict[coad_name][i][n] for n in reactant_names+product_names if n in Ncoad_energy_dict[coad_name][i].keys()))])
    energies = [get_reaction_energy_correction(Ncoad_energy_dict,ts_dict,ts_name,coad_name,i,Ncoad,reactant_names,product_names)*to_eV if Ncoad != 0 else 0.0 for Ncoad in Ncoads]
    plt.plot(Ncoads/MLsize,energies,color=cmap(i/L))

plt.legend(leg)
plt.ylabel("Reaction Energy Correction " + rstr + " [eV]")
plt.xlabel(r"$\theta_{"+coad_nice_name+r"}$")
#plt.ylim(-0.5,3.0)
#plt.xlim(0,0.65)

In [ ]:
#Fit activation barrier correction
L = len(Ncoad_energy_dict[coad_name])-1
Ncoads = np.array([0]+[Ncoad for Ncoad in sorted(Ncoad_energy_dict[coad_name][i][ts_name].keys()) if Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys() and all(Ncoad in Ncoad_energy_dict[coad_name][i][n] for n in reactant_names if n in Ncoad_energy_dict[coad_name][i].keys())])
coverages = Ncoads/MLsize
energies = np.array([get_barrier_correction(Ncoad_energy_dict,ts_dict,ts_name,coad_name,i,Ncoad,reactant_names)*to_eV if Ncoad != 0 else 0.0 for Ncoad in Ncoads])


params1 = np.polyfit(coverages,energies,1)
params2 = np.polyfit(coverages,energies,2)
params3 = np.polyfit(coverages,energies,3)
params4 = np.polyfit(coverages,energies,4)
params5 = np.polyfit(coverages,energies,5)
params6 = np.polyfit(coverages,energies,6)

vals1 = np.polyval(params1,coverages)
vals2 = np.polyval(params2,coverages)
vals3 = np.polyval(params3,coverages)
vals4 = np.polyval(params4,coverages)
vals5 = np.polyval(params5,coverages)
vals6 = np.polyval(params6,coverages)

plt.scatter(coverages,energies)
plt.plot(coverages,vals1)
plt.plot(coverages,vals2)
plt.plot(coverages,vals3)
plt.plot(coverages,vals4)
plt.plot(coverages,vals5)
plt.plot(coverages,vals6)
plt.legend(["SIDT","linear","2nd order","3rd order","4th order","5th order","6th order"])
plt.ylabel("Activation Barrier Correction " + rstr + " [eV]")
plt.xlabel(r"$\theta_{"+coad_nice_name+r"}$")

plt.figure()
poly_maes = [np.mean(np.abs(energies-vals1)),np.mean(np.abs(energies-vals2)),
                     np.mean(np.abs(energies-vals3)),np.mean(np.abs(energies-vals4)),
                    np.mean(np.abs(energies-vals5)),np.mean(np.abs(energies-vals6))]
plt.plot(range(1,7),poly_maes,'o-')

plt.xticks(range(1,7))
plt.ylim(0.0,np.max(poly_maes)+0.1*np.max(poly_maes))
plt.ylabel("Fitting MAE [eV]")
plt.xlabel("Polynomial Order")

#print(f"Polynomial coefficients (highest to lowest order): {params3}") #paramsN corresponds to the Nth order polynomial


In [ ]:
#analyze all adsorbates
L = len(Ncoad_energy_dict)-1
coad_name = "N#[Pt]"
leg = []
nice_names_dict = {} #dictionary mapping pynta names to desired names in plot
for config_name in Ncoad_energy_dict[coad_name][0].keys():
    if config_name in ts_dict.keys():
        continue
    if config_name in nice_names_dict.keys():
        leg.append(nice_names_dict[config_name])
    else:
        leg.append(config_name)
        
    Ncoads = np.array([0]+[Ncoad for Ncoad in sorted(Ncoad_energy_dict[coad_name][i][config_name].keys()) if Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys()])
    energies = [0.0]+[get_energy_correction_configuration(Ncoad_energy_dict,ts_dict,config_name,coad_name,L,Ncoad)*to_eV if Ncoad > 0 else 0.0 for Ncoad in Ncoads if Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys()]
    plt.plot(Ncoads/MLsize,energies)
    
for config_name in Ncoad_energy_dict[coad_name][0].keys():
    if config_name not in ts_dict.keys():
        continue
    if config_name in nice_names_dict.keys():
        leg.append(nice_names_dict[config_name])
    else:
        leg.append(config_name)
    forward = True
    if config_name in ts_dict.keys():
        if forward:
            reactant_names = ts_info[config_name]['species_names']
            product_names = ts_info[config_name]['reverse_names']
        else:
            reactant_names = ts_info[config_name]['reverse_names']
            product_names = ts_info[config_name]['species_names']
    Ncoads = np.array([0]+[Ncoad for Ncoad in sorted(Ncoad_energy_dict[coad_name][i][config_name].keys()) if Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys()])
    energies = [0.0]+[get_energy_correction_configuration(Ncoad_energy_dict,ts_dict,config_name,coad_name,L,Ncoad,reactant_names=reactant_names)*to_eV if Ncoad > 0 else 0.0 for Ncoad in Ncoads if Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys()]
    plt.plot(Ncoads/MLsize,energies)
    
plt.legend(leg)
plt.xlim(0,1)
plt.ylabel("Configuration Energy Correction [eV]")
plt.xlabel(r"$\theta_{"+coad_nice_name+r"}$")

In [ ]:
#analyze all adsorbates convergence
L = len(Ncoad_energy_dict[coad_name])-1
coad_name = "N#[Pt]"
leg = []

nice_names_dict = dict()
for config_name in Ncoad_energy_dict[coad_name][0].keys():
    if config_name in ts_dict.keys():
        continue
    if config_name in nice_names_dict.keys():
        leg.append(nice_names_dict[config_name])
    else:
        leg.append(config_name)
        
    Ncoads = np.array(sorted(Ncoad_energy_dict[coad_name][i][config_name].keys()))
    energies = [(get_energy_correction_configuration(Ncoad_energy_dict,ts_dict,config_name,coad_name,L,Ncoad)-get_energy_correction_configuration(Ncoad_energy_dict,ts_dict,config_name,coad_name,L-1,Ncoad))*to_eV if Ncoad > 0 else 0.0 for Ncoad in Ncoads if Ncoad+1 in Ncoad_energy_dict[coad_name][L][coad_name].keys()]
    plt.plot(np.array([N for N in Ncoads if N+1 in Ncoad_energy_dict[coad_name][L][coad_name].keys()])/MLsize,energies)
    
for config_name in Ncoad_energy_dict[coad_name][0].keys():
    if config_name not in ts_dict.keys():
        continue
    if config_name in nice_names_dict.keys():
        leg.append(nice_names_dict[config_name])
    else:
        leg.append(config_name)
    forward = True
    if config_name in ts_dict.keys():
        if forward:
            reactant_names = ts_info[config_name]['species_names']
            product_names = ts_info[config_name]['reverse_names']
        else:
            reactant_names = ts_info[config_name]['reverse_names']
            product_names = ts_info[config_name]['species_names']
    Ncoads = np.array(sorted(Ncoad_energy_dict[coad_name][i][config_name].keys()))
    energies = [(get_energy_correction_configuration(Ncoad_energy_dict,ts_dict,config_name,coad_name,L,Ncoad,reactant_names=reactant_names)-get_energy_correction_configuration(Ncoad_energy_dict,ts_dict,config_name,coad_name,L-1,Ncoad,reactant_names=reactant_names))*to_eV if Ncoad > 0 else 0.0 for Ncoad in Ncoads if Ncoad+1 in Ncoad_energy_dict[coad_name][L][coad_name].keys()]
    plt.plot(np.array([N for N in Ncoads if N+1 in Ncoad_energy_dict[coad_name][L][coad_name].keys()])/MLsize,energies)
    
plt.legend(leg)
plt.xlim(0,0.65)
plt.ylim(-1,1)
plt.ylabel(r"$E_{L}-E_{L-1}$ [eV]")
plt.xlabel(r"$\theta_{"+coad_nice_name+r"}$")

In [ ]:
#analyze all barriers
L = len(Ncoad_energy_dict)-1
coad_name = "N#[Pt]"
nice_names_dict = dict()
leg = []
for ts_name in ts_dict.keys():
    #analyze a reaction 

    forward = True
    if forward:
        reactant_names = ts_info[ts_name]['species_names']
        product_names = ts_info[ts_name]['reverse_names']
    else:
        reactant_names = ts_info[ts_name]['reverse_names']
        product_names = ts_info[ts_name]['species_names']
    
    rstr = "+".join(reactant_names) + "=>" + "+".join(product_names)
    Ncoads = np.array([Ncoad for Ncoad in sorted(Ncoad_energy_dict[coad_name][i][ts_name].keys()) if Ncoad == 0 or (Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys() and Ncoad in Ncoad_energy_dict[coad_name][i][ts_name].keys() and all(Ncoad in Ncoad_energy_dict[coad_name][i][n] for n in reactant_names if n in Ncoad_energy_dict[coad_name][i].keys()))])
    barrier_energies = [get_barrier_correction(Ncoad_energy_dict,ts_dict,ts_name,coad_name,L,Ncoad,reactant_names)*to_eV if Ncoad != 0 else 0.0 for Ncoad in Ncoads]
    plt.plot(Ncoads/MLsize,barrier_energies)
    if ts_name in nice_names_dict.keys():
        leg.append(nice_names_dict[ts_name])
    else:
        leg.append(rstr)
plt.legend(leg)
plt.ylabel("Barrier Correction [eV]")
plt.xlim(0,0.65)
plt.xlabel(r"$\theta_{"+coad_nice_name+r"}$")

In [ ]:
#analyze all reaction energies
L = len(Ncoad_energy_dict[coad_name])-1
coad_name = "N#[Pt]"
nice_names_dict = dict()
leg = []
for ts_name in ts_dict.keys():
    #analyze a reaction 

    forward = True
    if forward:
        reactant_names = ts_info[ts_name]['species_names']
        product_names = ts_info[ts_name]['reverse_names']
    else:
        reactant_names = ts_info[ts_name]['reverse_names']
        product_names = ts_info[ts_name]['species_names']
    
    rstr = "+".join(reactant_names) + "=>" + "+".join(product_names)
    Ncoads = np.array([Ncoad for Ncoad in sorted(Ncoad_energy_dict[coad_name][i][ts_name].keys()) if Ncoad == 0 or (Ncoad-1 in Ncoad_energy_dict[coad_name][i][coad_name].keys() and all(Ncoad in Ncoad_energy_dict[coad_name][i][n] for n in reactant_names+product_names if n in Ncoad_energy_dict[coad_name][i].keys()))])
    energies = [get_reaction_energy_correction(Ncoad_energy_dict,ts_dict,ts_name,coad_name,L,Ncoad,reactant_names,product_names)*to_eV if Ncoad != 0 else 0.0 for Ncoad in Ncoads]
    plt.plot(Ncoads/MLsize,energies)
    if ts_name in nice_names_dict.keys():
        leg.append(nice_names_dict[ts_name])
    else:
        leg.append(rstr)
plt.legend(leg)
plt.xlim(0,0.65)
plt.ylim(-2.0,2.0)
plt.ylabel("Reaction Energy Correction [eV]")
plt.xlabel(r"$\theta_{"+coad_nice_name+r"}$")